# Marginal Effects in Discrete Choice Models

This notebook covers four types of discrete choice models and shows how to compute and interpret marginal effects for each:

1. **Binary Choice (Logit vs Probit)** — AME, MEM, and comparison of the two families
2. **Discrete Change for Dummy Variables** — automatic detection of binary regressors
3. **Multinomial Logit** — ME as a matrix (variables x alternatives); zero-sum property
4. **Ordered Models (Logit/Probit)** — per-category ME; sign reversal; zero-sum across categories
5. **Fixed Effects Logit** — within-entity identification; effective sample loss
6. **Cross-Specification Comparison** — forest plots across Pooled Logit, Probit, and FE Logit
7. **Visualization Best Practices** — publication-ready forest plots with significance stars

**Table of Contents**

- [Section 1: Binary Choice — Logit vs Probit](#section-1)
- [Section 2: Discrete Change for Binary Explanatory Variables](#section-2)
- [Section 3: Multinomial Logit — Effects on Multiple Outcomes](#section-3)
- [Section 4: Ordered Models — Effects by Category](#section-4)
- [Section 5: Fixed Effects Logit — Within Effects](#section-5)
- [Section 6: Comparing Marginal Effects Across Specifications](#section-6)
- [Section 7: Visualization Best Practices for Discrete ME](#section-7)

**Prerequisites**: Notebook 01 (ME Fundamentals), knowledge of discrete choice models.

**Datasets**: Mroz (binary LFP), job_satisfaction (ordered), synthetic multinomial.

---
**Level**: Intermediate-Advanced | **Duration**: 90-120 min

In [ ]:
# Cell 2 — Setup
import sys
import os
sys.path.insert(0, '/home/guhaase/projetos/panelbox')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import warnings
warnings.filterwarnings('ignore')

# Core discrete models
from panelbox.models.discrete.binary import PooledLogit, PooledProbit, FixedEffectsLogit
from panelbox.models.discrete.ordered import OrderedLogit, OrderedProbit
from panelbox.models.discrete.multinomial import MultinomialLogit

# Marginal effects
from panelbox.marginal_effects.discrete_me import (
    compute_ame, compute_mem,
    compute_ordered_ame, compute_ordered_mem,
    MarginalEffectsResult, OrderedMarginalEffectsResult,
    _is_binary
)

# Utils from the tutorial series
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'marginal_effects'))
utils_path = '/home/guhaase/projetos/panelbox/examples/marginal_effects/utils'
sys.path.insert(0, utils_path)
from data_loaders import load_dataset
from me_helpers import format_me_table

# Output directories
os.makedirs('/home/guhaase/projetos/panelbox/examples/marginal_effects/outputs/plots', exist_ok=True)
os.makedirs('/home/guhaase/projetos/panelbox/examples/marginal_effects/outputs/tables', exist_ok=True)

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.float_format', '{:.4f}'.format)
print('Setup complete.')

## Section 1: Binary Choice — Logit vs Probit <a name="section-1"></a>

In binary choice models, the **marginal effect** of variable $x_k$ on $P(Y=1|X)$ is:

$$\text{ME}_k = \frac{\partial P(Y=1|X)}{\partial x_k} = \beta_k \cdot f(X'\beta)$$

where $f$ is the PDF of the assumed error distribution:
- **Logit**: $f(z) = \Lambda(z)[1 - \Lambda(z)]$, with $\Lambda$ the logistic CDF
- **Probit**: $f(z) = \phi(z)$, the standard normal PDF

**Key insight**: Logit coefficients are approximately 1.6–1.8× larger than Probit coefficients (due to scale normalization), but **AMEs are very similar** because the PDFs compensate for the scale difference.

Comparing AMEs from Logit and Probit is a standard robustness check.

In [ ]:
# Cell 4 — Load Mroz dataset and estimate Logit and Probit
#
# Mroz (1987): 753 married women. Outcome: inlf (in labor force)
# Note: column is 'inlf' (not 'lfp'), 'kidslt6' and 'kidsge6' (not kids_lt6/kids_618)

df = load_dataset('mroz')
print(f"Mroz dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nOutcome distribution (inlf):")
print(df['inlf'].value_counts())
print(f"\nSample statistics:")
print(df[['inlf', 'educ', 'age', 'kidslt6', 'kidsge6', 'nwifeinc']].describe())

# Add panel identifiers (cross-section treated as 1-period panel)
df = df.reset_index(drop=True)
df['id'] = range(len(df))
df['time'] = 1

# Formula using actual column names
formula = 'inlf ~ educ + age + kidslt6 + kidsge6 + nwifeinc'

# Fit both models
logit  = PooledLogit(formula, df, entity_col='id', time_col='time').fit(cov_type='nonrobust')
probit = PooledProbit(formula, df, entity_col='id', time_col='time').fit(cov_type='nonrobust')

print("\n" + "="*60)
print("POOLED LOGIT — Summary")
print("="*60)
print(logit.summary())

print("\n" + "="*60)
print("POOLED PROBIT — Summary")
print("="*60)
print(probit.summary())

In [ ]:
# Cell 5 — Compare AME: Logit vs Probit

# Exclude intercept from marginal effects computation
key_vars = ['educ', 'age', 'kidslt6', 'kidsge6', 'nwifeinc']

ame_logit  = compute_ame(logit,  varlist=key_vars)
ame_probit = compute_ame(probit, varlist=key_vars)

# Side-by-side comparison
comparison = pd.DataFrame({
    'Logit AME':  ame_logit.marginal_effects,
    'Probit AME': ame_probit.marginal_effects,
    'Difference': ame_logit.marginal_effects - ame_probit.marginal_effects,
    'Logit p':    ame_logit.pvalues,
    'Probit p':   ame_probit.pvalues,
})

print("=== AME Comparison: Logit vs Probit ===")
print(comparison.round(5))

# Coefficient ratio (Logit / Probit)
logit_params  = logit.params
probit_params = probit.params

# Filter to common variables
common_vars = [v for v in key_vars if v in logit_params.index and v in probit_params.index]
ratios = logit_params[common_vars] / probit_params[common_vars]
print(f"\nCoefficient ratios (Logit/Probit) — should be ~1.6-1.8:")
print(ratios.round(3))

print("\nNote: AMEs are very similar even though coefficients differ by ~1.6-1.8x.")
print("The scale difference is absorbed by the PDF evaluated at the linear predictor.")

## Section 2: Discrete Change for Binary Explanatory Variables <a name="section-2"></a>

For **continuous** variables, the marginal effect is the partial derivative $\partial P/\partial x_k$.

For **dummy (0/1) variables**, the derivative is not economically meaningful — instead, use the **discrete change**:

$$\text{DC}_k = P(Y=1 \mid x_k=1, X_{-k}=\bar{X}_{-k}) - P(Y=1 \mid x_k=0, X_{-k}=\bar{X}_{-k})$$

PanelBox automatically detects binary variables (using `_is_binary()`) and applies the discrete change formula for them. The output still appears in the same AME table.

**Example in this dataset**: `kidslt6` takes values 0, 1, 2, 3 — not a pure dummy, so the derivative is used. If we had `female` (0/1), the discrete change would be applied.

In [ ]:
# Cell 7 — Discrete Change Demonstration

print("=== Binary Variable Detection (_is_binary) ===")
print(f"{'Variable':<15} {'is_binary':<12} {'Unique values (first 6)'}")
print("-" * 55)

demo_vars = ['inlf', 'kidslt6', 'kidsge6', 'educ', 'age', 'nwifeinc']
for col in demo_vars:
    if col in df.columns:
        is_bin = _is_binary(df[col].values)
        unique_vals = sorted(df[col].unique())[:6]
        print(f"  {col:<13} {str(is_bin):<12} {unique_vals}")

print("\n--- Interpretation ---")
print("Variables detected as binary (0/1): use P(y=1|x=1) - P(y=1|x=0) [discrete change]")
print("Variables with more unique values  : use derivative formula ∂P/∂x")

# Manually demonstrate discrete change for a binary variable
# Create a synthetic binary version of kidslt6 for illustration
df_demo = df.copy()
df_demo['has_young_kid'] = (df_demo['kidslt6'] >= 1).astype(float)

print(f"\n'has_young_kid' is_binary: {_is_binary(df_demo['has_young_kid'].values)}")
print(f"Unique values: {sorted(df_demo['has_young_kid'].unique())}")

# Fit logit with binary dummy
formula_bin = 'inlf ~ educ + age + has_young_kid + kidsge6 + nwifeinc'
logit_bin = PooledLogit(formula_bin, df_demo, entity_col='id', time_col='time').fit(cov_type='nonrobust')
ame_bin = compute_ame(logit_bin, varlist=['educ', 'age', 'has_young_kid', 'kidsge6', 'nwifeinc'])

print("\n=== AME with binary dummy 'has_young_kid' ===")
print("(discrete change applied automatically for binary variables)")
df_ame = ame_bin.summary()
print(df_ame)

## Section 3: Multinomial Logit — Effects on Multiple Outcomes <a name="section-3"></a>

In Multinomial Logit with $J$ alternatives, the ME is a **matrix** of shape $(K \times J)$ (variables × alternatives). For variable $k$ and alternative $j$:

$$\frac{\partial P(Y=j|X)}{\partial x_k} = P(Y=j|X) \left[ \beta_{kj} - \sum_{m} P(Y=m|X) \beta_{km} \right]$$

**Key property — Zero-sum across alternatives**: for any variable $k$,
$$\sum_{j} \frac{\partial P(Y=j|X)}{\partial x_k} = 0$$

Increasing $x_k$ raises the probability of some alternatives and decreases others — the effects are zero-sum. This makes sense because probabilities must sum to 1.

In [ ]:
# Cell 9 — Generate Synthetic Multinomial Dataset (Transport Mode Choice)

np.random.seed(42)
N = 1000

income = np.random.normal(40, 15, N)    # income in thousands
age    = np.random.normal(38, 10, N)
urban  = np.random.binomial(1, 0.6, N).astype(float)

# Latent utilities for 3 alternatives (car=0, bus=1, bike=2)
u_car  =  0.02 * income + 0.01 * age + 0.5  * urban + np.random.gumbel(size=N)
u_bus  = -0.01 * income - 0.005* age - 0.3  * urban + np.random.gumbel(size=N)
u_bike = -0.005* income - 0.02 * age + 0.1  * urban + np.random.gumbel(size=N)

# Observed choice: argmax of utilities
choice = np.argmax(np.column_stack([u_car, u_bus, u_bike]), axis=1)

df_mnl = pd.DataFrame({
    'id': range(N), 'time': 1,
    'choice': choice.astype(float),
    'income': income,
    'age': age,
    'urban': urban
})

print("=== Synthetic Transport Mode Choice Dataset ===")
print(f"N = {N} observations, 3 alternatives: 0=Car, 1=Bus, 2=Bike")
print(f"\nMode shares:")
shares = df_mnl['choice'].value_counts(normalize=True).sort_index()
shares.index = shares.index.map({0.0: 'Car (0)', 1.0: 'Bus (1)', 2.0: 'Bike (2)'})
print(shares.round(3))
print(f"\nCovariate summary:")
print(df_mnl[['income', 'age', 'urban']].describe().round(2))

In [ ]:
# Cell 10 — Estimate Multinomial Logit and Compute ME Matrix
#
# MultinomialLogit takes: endog, exog, n_alternatives, base_alternative
# (does NOT use formula/PanelData interface — uses array interface)

endog_mnl = df_mnl['choice'].values
exog_mnl  = df_mnl[['income', 'age', 'urban']].values

mnl = MultinomialLogit(
    endog=endog_mnl,
    exog=exog_mnl,
    n_alternatives=3,
    base_alternative=0   # car is the reference category
)
mnl_result = mnl.fit()

print(mnl_result.summary())

# Compute ME matrix using the built-in method (AME: 'overall')
me_matrix = mnl_result.marginal_effects(at='overall')  # shape: (J, K)
se_matrix = mnl_result.marginal_effects_se(at='overall')

var_names = ['income', 'age', 'urban']
alt_names = ['Car (0)', 'Bus (1)', 'Bike (2)']

me_df = pd.DataFrame(me_matrix, index=alt_names, columns=var_names)
se_df = pd.DataFrame(se_matrix, index=alt_names, columns=var_names)

print("\n=== Multinomial AME Matrix (rows=alternatives, cols=variables) ===")
print(me_df.round(4))

# Verify zero-sum property (sum across alternatives for each variable = 0)
print("\nColumn sums (should be ~0 for each variable):")
print(me_df.sum(axis=0).round(8))
print("\nInterpretation: rows sum to ~0 because probabilities must add to 1.")

In [ ]:
# Cell 11 — Multinomial ME Heatmap

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

# Left: heatmap of ME matrix
ax = axes[0]
im = ax.imshow(me_df.values, cmap='RdBu_r', aspect='auto')
ax.set_xticks(range(me_df.shape[1]))
ax.set_xticklabels(me_df.columns, fontsize=10)
ax.set_yticks(range(me_df.shape[0]))
ax.set_yticklabels(me_df.index, fontsize=10)
ax.set_title("Multinomial Logit — AME Matrix\n(red=positive, blue=negative)")

# Annotate cells
for i in range(me_df.shape[0]):
    for j in range(me_df.shape[1]):
        ax.text(j, i, f"{me_df.iloc[i, j]:.4f}", ha='center', va='center',
                fontsize=9, color='black')

plt.colorbar(im, ax=ax, label="Average Marginal Effect")

# Right: bar chart of column sums (zero-sum verification)
ax2 = axes[1]
col_sums = me_df.sum(axis=0)
colors = ['tomato' if v < 0 else 'steelblue' for v in col_sums]
ax2.bar(var_names, col_sums.values, color=colors, alpha=0.8)
ax2.axhline(0, color='black', lw=1.5, ls='--')
ax2.set_title("Zero-Sum Check\n(column sums across alternatives)")
ax2.set_ylabel("Sum of ME across alternatives")
ax2.set_xlabel("Variable")

# Add value labels
for i, (v, lbl) in enumerate(zip(col_sums.values, var_names)):
    ax2.text(i, v + 0.0001 if v >= 0 else v - 0.0002, f"{v:.2e}",
             ha='center', va='bottom' if v >= 0 else 'top', fontsize=8)

plt.tight_layout()
plt.savefig('/home/guhaase/projetos/panelbox/examples/marginal_effects/outputs/plots/02_mnl_ame_heatmap.png',
            dpi=150, bbox_inches='tight')
plt.show()
print("Figure saved: 02_mnl_ame_heatmap.png")

## Section 4: Ordered Models — Effects by Category <a name="section-4"></a>

In **Ordered Logit/Probit** (e.g., satisfaction scale 1–5), the ME of variable $x_k$ varies across categories $j$:

$$\frac{\partial P(Y=j|X)}{\partial x_k} = \beta_k \left[ f(\kappa_{j-1} - X'\beta) - f(\kappa_j - X'\beta) \right]$$

where $\kappa$ are cutpoint parameters and $f$ is the PDF.

**Key property — Zero-sum across categories**: for each variable $k$,
$$\sum_j \frac{\partial P(Y=j|X)}{\partial x_k} = 0$$

**Implications**:
- A positive $\beta_k$ means $x_k$ shifts probability mass towards **higher** categories
- Middle categories can have positive or negative ME depending on the cutpoint spacing
- The lowest and highest categories always have **opposite signs**

**Ordered Logit vs Ordered Probit**: The ME should be similar — if they differ greatly, it may indicate model misspecification.

In [ ]:
# Cell 13 — Load Job Satisfaction Data
#
# Columns: satisfaction (1-5), age, female, educ, tenure, log_wage
# Note: column is 'log_wage' (not 'wage')

df_ord = load_dataset('job_satisfaction')
print(f"Dataset shape: {df_ord.shape}")
print(f"Columns: {list(df_ord.columns)}")

print("\nSatisfaction distribution (1=very dissatisfied, 5=very satisfied):")
sat_dist = df_ord['satisfaction'].value_counts().sort_index()
print(sat_dist)
print(f"\nPercentages:")
print((sat_dist / len(df_ord) * 100).round(1).astype(str) + '%')

print("\nSample statistics:")
print(df_ord[['satisfaction', 'log_wage', 'tenure', 'educ', 'female']].describe().round(3))

In [ ]:
# Cell 14 — Estimate Ordered Logit
#
# OrderedLogit takes: endog (array), exog (array), groups (entity IDs)
# It does NOT use the formula/PanelData interface
# Outcome must be 0-based integers; satisfaction is 1-5 → remap to 0-4

df_ord = df_ord.reset_index(drop=True)

# Remap satisfaction: 1-5 → 0-4
sat_raw = df_ord['satisfaction'].values.astype(int)
sat_vals = sat_raw - 1  # now 0-based

# Regressors (log_wage, tenure, educ, female) — using actual column names
exog_vars = ['log_wage', 'tenure', 'educ', 'female']
exog_ord = df_ord[exog_vars].values
groups_ord = np.arange(len(df_ord))  # cross-section, unique entity per row

ologit = OrderedLogit(
    endog=sat_vals,
    exog=exog_ord,
    groups=groups_ord
)

print("Fitting Ordered Logit... (may take a moment)")
ologit = ologit.fit(options={'disp': False})
print("Done.")

print(f"\nBeta coefficients (variables: {exog_vars}):")
for name, coef in zip(exog_vars, ologit.beta):
    print(f"  {name:<12}: {coef:+.4f}")

print(f"\nCutpoint parameters (thresholds between categories):")
for j, cp in enumerate(ologit.cutpoints):
    print(f"  kappa_{j+1}: {cp:.4f}  [between cat {j} and {j+1}]")

print(f"\nLog-likelihood: {ologit.llf:.4f}")
print(f"Converged: {ologit.converged}")

In [ ]:
# Cell 15 — Ordered AME
#
# compute_ordered_ame(model) takes the fitted model object (not result)
# Returns OrderedMarginalEffectsResult:
#   .marginal_effects : DataFrame  index=category_names, columns=variables
#   .std_errors       : DataFrame  same shape

# Attach exog_names so the result uses interpretable names
ologit.exog_names = exog_vars

ame_ord = compute_ordered_ame(ologit, varlist=exog_vars)

print("=== Ordered Logit AME by Category (rows=categories, cols=variables) ===")
print()

# Relabel categories from 'Category_0..4' to 'Sat.1..5'
cat_labels = [f'Sat.{i+1}' for i in range(ologit.n_categories)]
ame_ord.marginal_effects.index = cat_labels
ame_ord.std_errors.index = cat_labels

print("Marginal Effects (AME):")
print(ame_ord.marginal_effects.round(4))

print("\nStandard Errors:")
print(ame_ord.std_errors.round(4))

# Verify zero-sum property (sum across categories for each variable)
print("\nColumn sums (should be ~0 for each variable):")
print(ame_ord.marginal_effects.sum(axis=0).round(8))

verified = ame_ord.verify_sum_to_zero(tol=1e-6)
print(f"\nZero-sum verified (tol=1e-6): {verified}")

In [ ]:
# Cell 16 — Ordered ME Bar Plots by Category

# Categories from the result index; fallback to range if attribute missing
try:
    cats = list(ame_ord.marginal_effects.index)
except AttributeError:
    cats = [f'Cat.{i}' for i in range(ologit.n_categories)]

variables = list(ame_ord.marginal_effects.columns)
n_vars = len(variables)

fig, axes = plt.subplots(1, n_vars, figsize=(4 * n_vars, 4), sharey=False)
if n_vars == 1:
    axes = [axes]

for i, var in enumerate(variables):
    ax = axes[i]
    me_vals = ame_ord.marginal_effects[var].values
    se_vals = ame_ord.std_errors[var].values

    # Handle NaN SEs gracefully
    se_vals = np.where(np.isnan(se_vals), 0, se_vals)

    colors = ['tomato' if v < 0 else 'steelblue' for v in me_vals]
    x_pos = np.arange(len(cats))

    ax.bar(x_pos, me_vals, color=colors, alpha=0.75, width=0.6)
    ax.errorbar(x_pos, me_vals, yerr=1.96 * se_vals,
                fmt='none', color='black', capsize=4, lw=1.5)
    ax.axhline(0, color='black', lw=0.8)
    ax.set_title(f'AME of {var}', fontsize=11)
    ax.set_xlabel("Satisfaction Category")
    ax.set_xticks(x_pos)
    ax.set_xticklabels(cats, rotation=30, ha='right', fontsize=8)
    if i == 0:
        ax.set_ylabel("Average Marginal Effect")

plt.suptitle("Ordered Logit: AME by Category", y=1.02, fontsize=13)
plt.tight_layout()
plt.savefig('/home/guhaase/projetos/panelbox/examples/marginal_effects/outputs/plots/02_ordered_ame_by_category.png',
            dpi=150, bbox_inches='tight')
plt.show()
print("Figure saved: 02_ordered_ame_by_category.png")
print()
print("Interpretation:")
print("  log_wage > 0: higher wage → more likely to be very satisfied (Sat.5), less likely Sat.1")
print("  Signs for middle categories can go either way depending on cutpoint spacing")

In [ ]:
# Cell 17 — Compare Ordered Logit vs Ordered Probit

oprobit = OrderedProbit(
    endog=sat_vals,
    exog=exog_ord,
    groups=groups_ord
)

print("Fitting Ordered Probit...")
oprobit = oprobit.fit(options={'disp': False})
print("Done.")

# Attach variable names
oprobit.exog_names = exog_vars

ame_oprobit = compute_ordered_ame(oprobit, varlist=exog_vars)

# Relabel categories
ame_oprobit.marginal_effects.index = cat_labels
ame_oprobit.std_errors.index = cat_labels

print("\n=== Ordered Logit vs Ordered Probit AME — Highest Category (Sat.5) ===")
cat_top = cat_labels[-1]
cat_low = cat_labels[0]

comp = pd.DataFrame({
    'OLogit AME (Sat.5)':  ame_ord.marginal_effects.loc[cat_top],
    'OProbit AME (Sat.5)': ame_oprobit.marginal_effects.loc[cat_top],
    'Difference':          ame_ord.marginal_effects.loc[cat_top] - ame_oprobit.marginal_effects.loc[cat_top]
})
print(comp.round(5))

print(f"\n=== Lowest Category ({cat_low}) ===")
comp2 = pd.DataFrame({
    'OLogit AME (Sat.1)':  ame_ord.marginal_effects.loc[cat_low],
    'OProbit AME (Sat.1)': ame_oprobit.marginal_effects.loc[cat_low],
    'Difference':          ame_ord.marginal_effects.loc[cat_low] - ame_oprobit.marginal_effects.loc[cat_low]
})
print(comp2.round(5))

print("\nNote: Logit and Probit AMEs should be similar. Large differences suggest model sensitivity.")

## Section 5: Fixed Effects Logit — Within Effects <a name="section-5"></a>

**Fixed Effects Logit** (Chamberlain 1980) conditions out the individual fixed effect $\alpha_i$ using a conditional likelihood:

$$P\left(Y_{it} = 1 \mid \sum_t Y_{it}, X_i\right) = \frac{\exp\left(\sum_t Y_{it} X_{it}' \beta\right)}{\sum_{\mathbf{d}: \sum d_t = \sum_t Y_{it}} \exp\left(\sum_t d_t X_{it}' \beta\right)}$$

**Key properties**:
1. **Within interpretation**: ME measures the effect of a change in $x$ on $P(Y=1|\alpha_i, X)$, holding the individual effect constant
2. **Sample loss**: Individuals with no variation in $Y$ (always 0 or always 1) are dropped — they provide no information about $\beta$
3. **Consistent under strict exogeneity** even with arbitrary correlation between $\alpha_i$ and $X_i$

**Limitation**: Cannot estimate effects of time-invariant variables (e.g., gender), since they are absorbed by the fixed effect.

In [ ]:
# Cell 19 — FE Logit with Two-Period Panel
#
# FixedEffectsLogit requires genuine panel data (T >= 2)
# We create a synthetic 2-period panel from the Mroz cross-section
# Period 2 adds small random transitions in/out of the labor force

np.random.seed(123)
n_women = len(df)

# Create period 2 by applying small random transitions
transitions_in  = np.random.binomial(1, 0.08, n_women)  # some non-participants join
transitions_out = np.random.binomial(1, 0.08, n_women)  # some participants leave

inlf_t2 = df['inlf'].values.copy()
inlf_t2 = np.where(df['inlf'].values == 0, transitions_in,
                   np.where(transitions_out == 1, 0, inlf_t2))
inlf_t2 = np.clip(inlf_t2, 0, 1)

# Also add small changes in regressors over time (aging, children growing up)
df_t1 = df.copy()
df_t1['time'] = 1

df_t2 = df.copy()
df_t2['time'] = 2
df_t2['inlf']   = inlf_t2
df_t2['age']    = df['age']    + 1   # one year older
df_t2['kidslt6'] = np.clip(df['kidslt6'] - np.random.binomial(1, 0.15, n_women), 0, None)
df_t2['kidsge6'] = df['kidsge6'] + np.random.binomial(1, 0.15, n_women)

df_panel = pd.concat([df_t1, df_t2], ignore_index=True)
df_panel = df_panel.sort_values(['id', 'time']).reset_index(drop=True)

print(f"Panel dataset: {len(df_panel)} observations ({n_women} women × 2 periods)")
print(f"\nWithin-entity variation in inlf:")
variation = df_panel.groupby('id')['inlf'].nunique()
print(f"  Women with variation (switchers): {(variation > 1).sum()}")
print(f"  Women always in LF (no variation): {(df_panel.groupby('id')['inlf'].min() == 1).sum()}")
print(f"  Women never in LF (no variation):  {(df_panel.groupby('id')['inlf'].max() == 0).sum()}")

# Fit FE Logit
fe_formula = 'inlf ~ educ + age + kidslt6'
print(f"\nFitting Fixed Effects Logit: {fe_formula}")

try:
    fe_logit = FixedEffectsLogit(
        formula=fe_formula,
        data=df_panel,
        entity_col='id',
        time_col='time'
    ).fit(cov_type='nonrobust')

    print("\n" + "="*60)
    print("FIXED EFFECTS LOGIT — Summary")
    print("="*60)
    print(fe_logit.summary())

    # Compute AME from FE Logit
    fe_key_vars = ['educ', 'age', 'kidslt6']
    ame_fe = compute_ame(fe_logit, varlist=fe_key_vars)

    print("\n=== FE Logit AME (Within Effect) ===")
    print(ame_fe.summary())
    print("\nNote: Only within-entity variation (switchers) identifies these effects.")
    fe_success = True

except Exception as e:
    print(f"\nFE Logit failed: {e}")
    print("Falling back to Pooled Logit for comparison section.")
    fe_success = False
    ame_fe = None

## Section 6: Comparing Marginal Effects Across Specifications <a name="section-6"></a>

**Best practice in applied research**: estimate multiple specifications and compare their marginal effects. This demonstrates:
1. **Robustness** — if AMEs are similar, the finding is not model-dependent
2. **Sensitivity** — large differences may indicate omitted variable bias or heterogeneous effects
3. **Model selection** — FE Logit is consistent under endogenous fixed effects; Pooled Logit is not

Below we compare AMEs from Pooled Logit, Pooled Probit, and FE Logit for the same variables.

In [ ]:
# Cell 21 — Comparison Table: Logit, Probit, FE Logit

# Key variables available in all three models
key_vars_comp = ['educ', 'age', 'kidslt6']

# Gather AMEs (filter to key_vars)
ame_logit_comp  = ame_logit.marginal_effects.reindex(key_vars_comp)
ame_probit_comp = ame_probit.marginal_effects.reindex(key_vars_comp)

if fe_success and ame_fe is not None:
    # Only include variables present in FE result
    fe_available = [v for v in key_vars_comp if v in ame_fe.marginal_effects.index]
    ame_fe_comp = ame_fe.marginal_effects.reindex(key_vars_comp)
else:
    # If FE Logit failed, use placeholder NaNs
    ame_fe_comp = pd.Series([np.nan]*len(key_vars_comp), index=key_vars_comp)

comparison_table = pd.DataFrame({
    'Pooled Logit':  ame_logit_comp,
    'Pooled Probit': ame_probit_comp,
    'FE Logit':      ame_fe_comp
})

print("=== AME Comparison: Binary Models ===")
print(comparison_table.round(4))

# Forest plot for comparison
fig, axes = plt.subplots(1, len(key_vars_comp), figsize=(12, 4))
models_list   = ['Pooled Logit', 'Pooled Probit', 'FE Logit']
model_colors  = ['steelblue', 'tomato', 'forestgreen']

for j, var in enumerate(key_vars_comp):
    ax = axes[j]
    me_by_model = comparison_table.loc[var].values
    y_pos = np.arange(len(models_list))

    # Handle NaN for FE Logit
    valid_mask = ~np.isnan(me_by_model)
    ax.barh(y_pos[valid_mask], me_by_model[valid_mask],
            color=[c for c, v in zip(model_colors, valid_mask) if v],
            alpha=0.8, height=0.5)

    ax.set_yticks(list(y_pos))
    ax.set_yticklabels(models_list, fontsize=9)
    ax.axvline(0, color='black', lw=0.8)
    ax.set_title(f'AME of {var}', fontsize=11)
    ax.set_xlabel("Marginal Effect")

plt.suptitle("AME Comparison: Pooled Logit vs Probit vs FE Logit", fontsize=12)
plt.tight_layout()
plt.savefig('/home/guhaase/projetos/panelbox/examples/marginal_effects/outputs/plots/02_comparison_binary_models.png',
            dpi=150, bbox_inches='tight')
plt.show()
print("Figure saved: 02_comparison_binary_models.png")

## Section 7: Visualization Best Practices for Discrete ME <a name="section-7"></a>

**Choosing the right visualization**:

| Model type | Recommended plot |
|------------|------------------|
| Binary (single model) | Forest plot with 95% CIs |
| Binary (multiple models) | Side-by-side forest plot |
| Ordered | Bar plots per category (one subplot per variable) |
| Multinomial | Heatmap (alternatives × variables) |

**Guidelines**:
- Always include **confidence intervals** ($\pm 1.96 \times \text{SE}$)
- Use **color coding**: blue = positive, red = negative
- Include **significance stars**: $^*p < .05$, $^{**}p < .01$, $^{***}p < .001$
- Label axes in **probability units** (pp) or fractions
- Report the method (AME vs MEM) and any restrictions applied

In [ ]:
# Cell 23 — Publication-Ready Forest Plot (Pooled Logit)

fig, ax = plt.subplots(figsize=(8, 5))

me_vals  = ame_logit.marginal_effects.values
se_vals  = ame_logit.std_errors.values
labels   = list(ame_logit.marginal_effects.index)
y_pos    = np.arange(len(labels))
pvals    = ame_logit.pvalues.values

def significance_stars(p):
    if np.isnan(p):   return ''
    if p < 0.001: return '***'
    if p < 0.01:  return '**'
    if p < 0.05:  return '*'
    return ''

colors     = ['tomato' if v < 0 else 'steelblue' for v in me_vals]
sig_stars  = [significance_stars(p) for p in pvals]

bars = ax.barh(y_pos, me_vals, xerr=1.96 * se_vals,
               color=colors, alpha=0.75, capsize=5, height=0.5,
               error_kw={'elinewidth': 1.5, 'capthick': 1.5})

# Significance stars
for i, (v, sig, se) in enumerate(zip(me_vals, sig_stars, se_vals)):
    if sig:
        x_star = v + 1.96*se + 0.002 if v >= 0 else v - 1.96*se - 0.002
        ha = 'left' if v >= 0 else 'right'
        ax.text(x_star, i, sig, ha=ha, va='center', fontsize=11,
                color='black', fontweight='bold')

# Zero reference line
ax.axvline(0, color='black', lw=1.0, ls='--', alpha=0.7)

# Labels
ax.set_yticks(y_pos)
ax.set_yticklabels(labels, fontsize=11)
ax.set_xlabel("Average Marginal Effect on P(inlf=1)", fontsize=11)
ax.set_title(
    "AME — Pooled Logit: Women's Labor Force Participation\n"
    "Mroz (1987) | 95% Confidence Intervals\n"
    "Significance: * p<.05  ** p<.01  *** p<.001",
    fontsize=11
)

# Legend for colors
import matplotlib.patches as mpatches
pos_patch = mpatches.Patch(color='steelblue', alpha=0.75, label='Positive effect')
neg_patch = mpatches.Patch(color='tomato',    alpha=0.75, label='Negative effect')
ax.legend(handles=[pos_patch, neg_patch], loc='lower right', fontsize=9)

plt.tight_layout()
plt.savefig('/home/guhaase/projetos/panelbox/examples/marginal_effects/outputs/plots/02_publication_forest_plot.png',
            dpi=150, bbox_inches='tight')
plt.show()
print("Figure saved: 02_publication_forest_plot.png")

## Key Takeaways

1. **Binary models**: AME is nearly identical between Logit and Probit despite coefficient differences of ~1.6-1.8x — use both as a robustness check.

2. **Dummy variables**: Always use **discrete change** (not derivative) for 0/1 regressors. PanelBox detects binary variables automatically via `_is_binary()`.

3. **Multinomial Logit**: ME is a **matrix** ($J \times K$); for any variable $k$, the sum of ME across all alternatives $j$ equals **zero**. Increasing $x_k$ creates a zero-sum redistribution of probability mass.

4. **Ordered models**: ME is a **vector** (one value per category); effects must sum to **zero across categories**. Positive $\beta_k$ shifts probability toward higher categories, but middle categories can have positive or negative ME.

5. **FE Logit**: Only within-entity variation identifies the parameters; individuals with no variation in $Y$ are dropped. Interpret as **within-person** effects, controlling for time-invariant unobservables.

6. **Comparison**: Reporting ME from multiple specifications demonstrates robustness. If Pooled Logit and FE Logit AMEs differ substantially, there may be unobserved heterogeneity biasing the pooled estimates.

---

**Bridge to Notebook 03**: Count data models (Poisson, Negative Binomial) have a different functional form — $E[Y|X] = \exp(X'\beta)$ — but the same ME logic applies. Marginal effects are $\partial E[Y]/\partial x_k = \beta_k \cdot \exp(X'\beta)$. Incidence Rate Ratios (IRR = $\exp(\beta_k)$) offer a multiplicative interpretation. See `03_count_me.ipynb`.

In [ ]:
# Cell 25 — Export All Results

out_tables = '/home/guhaase/projetos/panelbox/examples/marginal_effects/outputs/tables'

# Binary model AME tables
try:
    fmt_logit  = format_me_table(ame_logit)
    fmt_probit = format_me_table(ame_probit)
    fmt_logit.to_csv(f"{out_tables}/02_ame_logit.csv", index=False)
    fmt_probit.to_csv(f"{out_tables}/02_ame_probit.csv", index=False)
    print("Saved: 02_ame_logit.csv, 02_ame_probit.csv")
except Exception as e:
    print(f"Warning (binary tables): {e}")
    ame_logit.marginal_effects.to_frame('AME').to_csv(f"{out_tables}/02_ame_logit.csv")
    ame_probit.marginal_effects.to_frame('AME').to_csv(f"{out_tables}/02_ame_probit.csv")
    print("Saved (fallback): 02_ame_logit.csv, 02_ame_probit.csv")

# Comparison table
comparison_table.to_csv(f"{out_tables}/02_comparison_binary.csv")
print("Saved: 02_comparison_binary.csv")

# Ordered AME
ame_ord.marginal_effects.to_csv(f"{out_tables}/02_ordered_ame.csv")
ame_ord.std_errors.to_csv(f"{out_tables}/02_ordered_se.csv")
print("Saved: 02_ordered_ame.csv, 02_ordered_se.csv")

# Multinomial AME
me_df.to_csv(f"{out_tables}/02_multinomial_ame.csv")
print("Saved: 02_multinomial_ame.csv")

print("\nAll results saved to outputs/tables/")
print("\n" + "="*60)
print("Notebook 02 complete!")
print("="*60)
print("Summary of outputs:")
print("  Plots : 02_mnl_ame_heatmap.png")
print("          02_ordered_ame_by_category.png")
print("          02_comparison_binary_models.png")
print("          02_publication_forest_plot.png")
print("  Tables: 02_ame_logit.csv")
print("          02_ame_probit.csv")
print("          02_comparison_binary.csv")
print("          02_ordered_ame.csv")
print("          02_multinomial_ame.csv")